In [2]:
from flask import Flask, render_template
from markupsafe import escape

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from sqlalchemy.sql import func
import os

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(app.root_path, '../douban.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 
app.app_context().push()

db = SQLAlchemy(app)



In [3]:
movie_actor_association = db.Table(
    'movie_actor_association',
    db.Column('movie_id', db.Integer, db.ForeignKey('douban_movies.movie_id')),
    db.Column('person_id', db.Integer, db.ForeignKey('persons.person_id')),
)


class DoubanMovie(db.Model):
    __tablename__ = 'douban_movies'

    movie_id = db.Column(db.Integer, primary_key=True)
    movie_name = db.Column(db.String(20))
    release_date = db.Column(db.String(15))
    country = db.Column(db.String(20))
    movie_type = db.Column(db.String(10))
    release_year = db.Column(db.Integer)
    description = db.Column(db.Text)
    douban_url = db.Column(db.String(255))
    poster  = db.Column(db.String(255))
    douban_rate  = db.Column(db.Float)
    rating_count = db.Column(db.Integer)
    
    # actors = db.relationship('Person', secondary=movie_actor_association, backref='movies', lazy='dynamic')
    persons = db.relationship('Person', secondary=movie_actor_association, backref='movies', lazy='dynamic')
 
    def __repr__(self):
        return f'<Douban {self.movie_name}>' 
    
class Person(db.Model):
    __tablename__ = 'persons'

    person_id = db.Column(db.Integer, primary_key=True)
    person_name = db.Column(db.String(20))
    person_name_en = db.Column(db.String(20))
    gender = db.Column(db.String(2))
    birth_date = db.Column(db.String(20))
    birth_place = db.Column(db.String(20))
    birth_year = db.Column(db.Integer())
    
    def __repr__(self):
        return f'<Person {self.person_name}>' 

In [13]:
movies = DoubanMovie.query.all()

In [14]:
import pandas as pd

In [15]:
movies = pd.DataFrame(movies)

In [16]:
movies

,0
0,<Douban 哈利·波特与阿兹卡班的囚徒>
1,<Douban 大鱼>
2,<Douban 霸王别姬>
3,<Douban 放牛班的春天>
4,<Douban 指环王3：王者无敌>
...,...
423,<Douban 阳光普照>
424,<Douban 利刃出鞘>
425,<Douban 黑水>
426,<Douban 调音师>


In [11]:
vars = {}
data = movies.loc[
    movies.country.str.contains('大陆'), ['movie_name', 'douban_rate']
].sort_values('douban_rate', ascending=False)[:10].set_index('movie_name')
vars['echart_mainland'] = hbar_option("大陆电影高分榜", data.index.tolist(), data['douban_rate'].tolist())

AttributeError: 'DataFrame' object has no attribute 'country'